In [1]:
import sys
import gc
import re
import json
import _pickle as p
import pandas as pd
import numpy as np
from importlib import reload
from IPython.display import display, HTML

sys.path.append('..')
sys.path.append('../lib')
import stats as S
S = reload(S)

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
display(HTML("<style>.container{width: 100%}</style>"))

In [2]:
lemm = S.lemm_h
def freq_dict(text, suff='ph'):
    txt_lemm = lemm(text)
    kwds = S.keywords(txt_lemm)
    word_cnt = len(kwds)
    word_cnt_log_inv = 1 / np.log2(word_cnt)
    kwds_sorted = sorted(kwds.items(), key=lambda x: -x[1])
    kwds_inverted = {i: word_cnt_log_inv/v for i, v in kwds.items()}
    v = kwds_inverted.values()
    with open(f'{suff}-wordimp.json', 'w') as f:
        f.write(json.dumps(kwds_inverted))
    print(f"Stats: words count: {word_cnt}, log_inv = {word_cnt_log_inv}; min = {min(v)}, max = {max(v)}")
    return kwds_inverted

def prepare_dataset(df, ds_basename, threshold=1e-2, max_len=None):
    Targ = 'Targ'
    Lemm = 'Lemm'
    Kw = 'Kw'
    KwMult = 'KwMult'
    KwFilt = 'KwFilt'
    FSum = 'Fsum'
    KwNorm = 'KwNorm'
    txt = "\n".join(df.loc[:, Targ].apply(str))
    kwds_inverted = freq_dict(txt, suff=ds_basename)
    df = df.loc[df.loc[:, Targ].notnull()]
    if max_len is not None:
        df = df.head(max_len)
    df.loc[:, Lemm] = df.loc[:, Targ].apply(lemm)
    df.loc[:, Kw] = df.loc[:, Lemm].apply(lambda x: {i: 1 for i, v in S.keywords(x).items()})
    df.loc[:, KwMult] = df.loc[:, Kw].apply(lambda x: S.multiply2stats(kwds_inverted, x, max_dist=0))
    df.loc[:, KwFilt] = df.loc[:, KwMult].apply(lambda x: {i: v for i, v in x.items() if v > max(x.values())*threshold})
    df.loc[:, FSum] = df.loc[:, KwFilt].apply(lambda x: sum(x.values()))
    df.loc[:, KwNorm] = df.apply(lambda x: {i: v/x[FSum] for i, v in x[KwFilt].items()}, axis=1)
    
    df.drop([Lemm, Kw, KwFilt, FSum], axis=1, inplace=True)
    
    df.to_csv(f'{ds_basename}-kw-thr{threshold:.6f}.csv')
    gc.collect()
    return df

In [29]:
pharma.head()

,PharmaLink-href,PharmaName,ActiveSubstance,Targ,AnaloguesText,AnalogsLinks,Lots-href,Аптека/Адрес,Метро,Цена
0,https://www.rlsnet.ru/tn_index_id_3684.htm,Энап®-Н (Enap®-H),Гидрохлоротиазид + Эналаприл (Hydrochlorothiazide + Enalapril),"Фармакологическое действие — диуретическое, гипотензивное",NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.rlsnet.ru/tn_index_id_29955.htm,Энзикс дуо форте (Enzix duo forte),Индапамид + Эналаприл (Indapamide + Enalapril),Фармакологическое действие — гипотензивное,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.rlsnet.ru/tn_index_id_29592.htm,Элефлокс (Eleflox®),Левофлоксацин* (Levofloxacin),"Фармакологическое действие — бактерицидное, антибактериальное широкого спектра",NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.rlsnet.ru/tn_index_id_3682.htm,Энап® (Enap®),Эналаприл* (Enalapril*),Фармакологическое действие — гипотензивное,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.rlsnet.ru/tn_index_id_66219.htm,Эдарби® (Edarbi),Азилсартана медоксомил* (Azilsartan medoxomil*),"Фармакологическое действие — гипотензивное, блокирующее АТ1-рецепторы ангиотензина II",NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pharma.groupby('PharmaName').agg({''})

In [3]:
# articles = pd.read_csv('medical_site.csv')
pharma = pd.read_csv('initial/pharma.csv')
pharma.drop(['web-scraper-order', 'web-scraper-start-url', 'Page', 'Page-href', 'PharmaLink', 'AnalogsLinks-href', 'Lots', 'Instruction'], axis=1, inplace=True)
pharma.rename(columns={"PharmacologicAction": "Targ"}, inplace=True)

In [4]:
pattern = re.compile('[^\w\d\s]+') 
sdict = pd.read_csv('initial/def.csv')
sdict.drop(['web-scraper-order', 'web-scraper-start-url', 'Pages', 'Pages-href'], axis=1, inplace=True)
sdict.rename(columns={"Definitions": "Targ"}, inplace=True)

In [5]:
pdict = pd.read_csv('initial/prof.csv')
pdict.drop(['web-scraper-order', 'web-scraper-start-url', 'ProfLinks-href'], axis=1, inplace=True)
pdict.drop([1, 3, 4, 5, 11, 16, 26, 27, 33], axis=0, inplace=True)
pdict.rename(columns={"DescriptionText": "Targ"}, inplace=True)

In [26]:
pattern = re.compile('[^\w\d\s]+') 
sdict2 = pd.read_csv('initial/def2.csv', converters={'Description': str})
sdict2.drop(['web-scraper-order', 'web-scraper-start-url', 'Disease-href'], axis=1, inplace=True)
sdict2.rename(columns={"Description": "Targ"}, inplace=True)
sdict2 = sdict2.groupby('Disease').agg({'Targ': 'sum'}).reset_index(drop=False)

In [18]:
sdict2.head()

Targ
Disease                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [15]:
sdict2[sdict2['Disease'] == 'Боль головная']

,Disease,Targ
2,Боль головная,"Голова болит при повышенном кровяном давлении, почечной недостаточности, также болезнях почек, желудка и печени, лихорадке, вызванной инфекционными заболеваниями."
528,Боль головная,
733,Боль головная,"Большинство пациентов, постоянно жалующихся на то, что часто болит голова, страдают от душевного напряжения, неприятных и тяжелых переживаний."
1537,Боль головная,"Причина боли - не в изменении самого вещества головного мозга (дело в том, что в нем нет рецепторов боли, поэтому человек и не чувствует ее). Болевые ощущения могут вызвать находящиеся в черепе кровеносные сосуды и нервы, мягкая и твердая оболочки головного мозга, большие кровеносные сосуды основания черепа. Однако в таких случаях человеку всегда кажется, что у него болит кожа, покрывающая череп."
1670,Боль головная,"Голова может начать болеть вследствие внешних факторов, например, внезапное изменение погоды, жара, сквозняки, знойные солнечные лучи, химические вещества, длительное пребывании в закрытом помещении."
2124,Боль головная,
2415,Боль головная,"При длительных и частых головных болях необходимо обращаться к врачу. Когда часто болит голова, это всегда является предупреждающим сигналом!"
2701,Боль головная,
2977,Боль головная,
3276,Боль головная,"Голова почти всегда болит при заболеваниях органов, расположенных в области головы, например, глаз, носа, околоносовых пазух, ушей, зубов."


In [ ]:
Ph = prepare_dataset(pharma, 'pharma', max_len=None)
Sd = prepare_dataset(sdict, 'def', max_len=None)
Sd2 = prepare_dataset(sdict2, 'def2', max_len=None)
Pd = prepare_dataset(pdict, 'prof', max_len=None)

In [21]:
display(pharma.head())
display(Sdict.head())
display(Pdict.head())

,PharmaLink-href,PharmaName,ActiveSubstance,Targ,AnaloguesText,AnalogsLinks,Lots-href,Аптека/Адрес,Метро,Цена,KwMult,KwNorm
0,https://www.rlsnet.ru/tn_index_id_3684.htm,Энап®-Н (Enap®-H),Гидрохлоротиазид + Эналаприл (Hydrochlorothiazide + Enalapril),"Фармакологическое действие — диуретическое, гипотензивное",NaN,NaN,NaN,NaN,NaN,NaN,"{'фармакологическ': 6.792152992797087e-06, 'действ': 6.792152992797087e-06, 'диуретическ': 0.00017918572329740808, 'гипотензивн': 5.5395931727523396e-05}","{'фармакологическ': 0.02736939814444982, 'действ': 0.02736939814444982, 'диуретическ': 0.7220398904336769, 'гипотензивн': 0.22322131327742342}"
1,https://www.rlsnet.ru/tn_index_id_29955.htm,Энзикс дуо форте (Enzix duo forte),Индапамид + Эналаприл (Indapamide + Enalapril),Фармакологическое действие — гипотензивное,NaN,NaN,NaN,NaN,NaN,NaN,"{'фармакологическ': 6.792152992797087e-06, 'действ': 6.792152992797087e-06, 'гипотензивн': 5.5395931727523396e-05}","{'фармакологическ': 0.09846520130946812, 'действ': 0.09846520130946812, 'гипотензивн': 0.8030695973810636}"
2,https://www.rlsnet.ru/tn_index_id_29592.htm,Элефлокс (Eleflox®),Левофлоксацин* (Levofloxacin),"Фармакологическое действие — бактерицидное, антибактериальное широкого спектра",NaN,NaN,NaN,NaN,NaN,NaN,"{'фармакологическ': 6.792152992797087e-06, 'действ': 6.792152992797087e-06, 'бактерицидн': 0.00020109244181966174, 'антибактериальн': 8.757472892897214e-05, 'широк': 0.00012360017136892504, 'спектр': 0.00012360017136892504}","{'фармакологическ': 0.012361689873596366, 'действ': 0.012361689873596366, 'бактерицидн': 0.3659873981541721, 'антибактериальн': 0.15938563824052, 'широк': 0.2249517919290576, 'спектр': 0.2249517919290576}"
3,https://www.rlsnet.ru/tn_index_id_3682.htm,Энап® (Enap®),Эналаприл* (Enalapril*),Фармакологическое действие — гипотензивное,NaN,NaN,NaN,NaN,NaN,NaN,"{'фармакологическ': 6.792152992797087e-06, 'действ': 6.792152992797087e-06, 'гипотензивн': 5.5395931727523396e-05}","{'фармакологическ': 0.09846520130946812, 'действ': 0.09846520130946812, 'гипотензивн': 0.8030695973810636}"
4,https://www.rlsnet.ru/tn_index_id_66219.htm,Эдарби® (Edarbi),Азилсартана медоксомил* (Azilsartan medoxomil*),"Фармакологическое действие — гипотензивное, блокирующее АТ1-рецепторы ангиотензина II",NaN,NaN,NaN,NaN,NaN,NaN,"{'фармакологическ': 6.792152992797087e-06, 'действ': 6.792152992797087e-06, 'гипотензивн': 5.5395931727523396e-05, 'блокирова': 0.0002990605545010354, 'ат1': 0.00041927116954556924, 'рецептор': 0.00041927116954556924, 'ангиотензин': 0.00041927116954556924, 'ii': 0.00041927116954556924}","{'фармакологическ': 0.0033211424389920133, 'действ': 0.0033211424389920133, 'гипотензивн': 0.02708681326861837, 'блокирова': 0.14623090799561805, 'ат1': 0.2050099984644449, 'рецептор': 0.2050099984644449, 'ангиотензин': 0.2050099984644449, 'ii': 0.2050099984644449}"


,Targ,KwMult,KwNorm
0,"Плазма - жидкая часть крови, в которой находятся форменные элементы (эритроциты, лейкоциты, тромбоциты). По изменениям в составе плазмы крови диагностируются различные заболевания (ревматизм, сахарный диабет и тд.). Из плазмы крови готовят лекарственные препараты.","{'плазм': 0.007661231513977942, 'жидк': 0.008427354665375735, 'част': 0.001053419333171967, 'кров': 0.0005333768775554263, 'котор': 0.0008687994500387356, 'наход': 0.00936372740597304, 'формен': 0.02106838666343934, 'элемент': 0.003664067245815537, 'эритроц': 0.004213677332687868, 'лейкоц': 0.005618236443583824, 'тромбоц': 0.01404559110895956, 'изменен': 0.006019539046696954, 'соста': 0.004435449823881966, 'диагностирова': 0.08427354665375736, 'различн': 0.00234093185149326, 'заболеван': 0.0004506606772928201, 'ревматизм': 0.01404559110895956, 'сахарн': 0.007661231513977942, 'диабет': 0.007661231513977942, 'тд': 0.006019539046696954, 'готов': 0.04213677332687868, 'лекарствен': 0.003009769523348477, 'препарат': 0.0026335483329299175}","{'плазм': 0.02990079771208049, 'жидк': 0.03289087748328854, 'част': 0.0041113596854110675, 'котор': 0.003390812111679231, 'наход': 0.036545419425876156, 'формен': 0.08222719370822135, 'элемент': 0.014300381514473277, 'эритроц': 0.01644543874164427, 'лейкоц': 0.021927251655525695, 'тромбоц': 0.05481812913881423, 'изменен': 0.023493483916634673, 'соста': 0.017310988149099232, 'диагностирова': 0.3289087748328854, 'различн': 0.009136354856469039, 'ревматизм': 0.05481812913881423, 'сахарн': 0.02990079771208049, 'диабет': 0.02990079771208049, 'тд': 0.023493483916634673, 'готов': 0.1644543874164427, 'лекарствен': 0.011746741958317337, 'препарат': 0.010278399213527668}"
1,"Контрактура - ограничение подвижности сустава, вызванное рубцовым стягиванием кожи, сухожилий, заболеваниями мышц, сустава, болевым рефлексом, др. причинами.","{'заболеван': 0.0004506606772928201, 'контрактур': 0.04213677332687868, 'ограничен': 0.01404559110895956, 'подвижн': 0.00936372740597304, 'суста': 0.004013026031131303, 'вызыва': 0.0014529921836854717, 'рубцов': 0.08427354665375736, 'стягиван': 0.08427354665375736, 'кож': 0.0012039078093393908, 'сухожил': 0.01053419333171967, 'мышц': 0.0017198682990562726, 'болев': 0.02809118221791912, 'рефлекс': 0.012039078093393909, 'др': 0.0006532833073934679, 'причин': 0.0029059843673709435}","{'контрактур': 0.14232827874485357, 'ограничен': 0.04744275958161785, 'подвижн': 0.03162850638774524, 'суста': 0.01355507416617653, 'вызыва': 0.0049078716808570195, 'рубцов': 0.28465655748970714, 'стягиван': 0.28465655748970714, 'кож': 0.004066522249852959, 'сухожил': 0.03558206968621339, 'мышц': 0.005809317499789941, 'болев': 0.0948855191632357, 'рефлекс': 0.04066522249852959, 'причин': 0.009815743361714039}"
2,"Сердечный толчок - вызванное сокращением сердца сотрясение передней стенки грудной клетки, распространяющееся в подложечную область. Сотрясение в области верхушки сердца называется верхушечным толчком. Их смещение, расширение площади, изменение силы помогают при диагностике заболеваний сердца.","{'изменен': 0.006019539046696954, 'заболеван': 0.0004506606772928201, 'вызыва': 0.0014529921836854717, 'сердечн': 0.003009769523348477, 'толчок': 0.01404559110895956, 'сокращен': 0.004013026031131303, 'сердц': 0.001755698888619945, 'сотрясен': 0.04213677332687868, 'передн': 0.008427354665375735, 'стенк': 0.003830615756988971, 'грудн': 0.00936372740597304, 'клетк': 0.0006196584312776277, 'распространя': 0.008427354665375735, 'подложечн': 0.08427354665375736, 'област': 0.00351139777723989, 'верхушк': 0.08427354665375736, 'называ': 0.00702279555447978, 'верхушечн': 0.08427354665375736, 'их': 0.001478483274627322, 'смещен': 0.04213677332687868, 'расширен': 0.008427354665375735, 'площад': 0.08427354665375736, 'сил': 0.02809118221791912, 'помога': 0.02809118221791912, 'диагностик': 0.01053419333171967}","{'изменен': 0.010581572478332685, 'вызыва': 0.0025541726671837513, 'сердечн': 0.0052907862391663426, 'толчок': 0.02469033

,ProfLinks,Targ,KwMult,KwNorm
0,Диетолог,"Дието́лог — специалист в области диетологии, соответствующий академическим и профессиональным требованиям. Врач-диетолог является дипломированным врачом, закончившим медицинский вуз и получившим затем дополнительное образование по диетологии, подтвержденное соответствующим дипломом. Является экспертом в области питания. Врачи-диетологи занимаются лечением ожирения, а также дают рекомендации пациентам по вопросам правильного питания, ведения здорового образа жизни, либо ставят цели для достижения определённых результатов, связанных со здоровьем.","{'диет': 0.11444145590234722, 'лог': 0.016348779414621033, 'специалист': 0.006357858661241512, 'област': 0.012715717322483024, 'диетолог': 0.028610363975586806, 'соответств': 0.028610363975586806, 'академическ': 0.11444145590234722, 'профессиональн': 0.05722072795117361, 'требован': 0.11444145590234722, 'врач': 0.0036916598678176523, 'явля': 0.016348779414621033, 'дипломирова': 0.11444145590234722, 'заканчива': 0.11444145590234722, 'медицинск': 0.004238572440827675, 'вуз': 0.11444145590234722, 'получа': 0.016348779414621033, 'зат': 0.11444145590234722, 'дополнительн': 0.11444145590234722, 'образован': 0.007152590993896701, 'подтвержда': 0.11444145590234722, 'дипл': 0.05722072795117361, 'эксперт': 0.05722072795117361, 'питан': 0.05722072795117361, 'занима': 0.016348779414621033, 'лечен': 0.014305181987793403, 'ожирен': 0.11444145590234722, 'такж': 0.05722072795117361, 'дава': 0.11444145590234722, 'рекомендац': 0.11444145590234722, 'пациент': 0.016348779414621033, 'вопрос': 0.038147151967449076, 'правильн': 0.11444145590234722, 'веден': 0.11444145590234722, 'здоров': 0.038147151967449076, 'образ': 0.11444145590234722, 'жизн': 0.05722072795117361, 'либ': 0.05722072795117361, 'став': 0.11444145590234722, 'цел': 0.11444145590234722, 'достижен': 0.11444145590234722, 'определен': 0.05722072795117361, 'результат': 0.11444145590234722, 'связыва': 0.05722072795117361}","{'диет': 0.03875305962069591, 'лог': 0.00553615137438513, 'специалист': 0.0021529477567053283, 'област': 0.0043058955134106566, 'диетолог': 0.009688264905173978, 'соответств': 0.009688264905173978, 'академическ': 0.03875305962069591, 'профессиональн': 0.019376529810347955, 'требован': 0.03875305962069591, 'врач': 0.0012500986974418036, 'явля': 0.00553615137438513, 'дипломирова': 0.03875305962069591, 'заканчива': 0.03875305962069591, 'медицинск': 0.0014352985044702189, 'вуз': 0.03875305962069591, 'получа': 0.00553615137438513, 'зат': 0.03875305962069591, 'дополнительн': 0.03875305962069591, 'образован': 0.0024220662262934944, 'подтвержда': 0.03875305962069591, 'дипл': 0.019376529810347955, 'эксперт': 0.019376529810347955, 'питан': 0.019376529810347955, 'занима': 0.00553615137438513, 'лечен': 0.004844132452586989, 'ожирен': 0.03875305962069591, 'такж': 0.019376529810347955, 'дава': 0.03875305962069591, 'рекомендац': 0.03875305962069591, 'пациент': 0.00553615137438513, 'вопрос': 0.012917686540231971, 'правильн': 0.03875305962069591, 'веден': 0.03875305962069591, 'здоров': 0.012917686540231971, 'образ': 0.03875305962069591, 'жизн': 0.019376529810347955, 'либ': 0.019376529810347955, 'став': 0.03875305962069591, 'цел': 0.03875305962069591, 'достижен': 0.03875305962069591, 'определен': 0.019376529810347955, 'результат': 0.03875305962069591, 'связыва': 0.019376529810347955}"
2,Акушерка,"Акуше́рка (фр. accoucher — помогать при родах[1]) — средний медицинский работник, специалист в акушерстве, оказывающий помощь беременным и роженицам (во время родов). В России для работы акушеркой необходимо получить среднее специальное образование по специальности «Медико-акушерское дело». Квалификации: «акушер(ка)», «фельдшер-акушер». Не следует путать с врачом акушер-гинекологом (высшее медицинское образование).","{'специалист': 0.006357858661241512, 'врач': 0.0036916598678176523, 'медицинск': 0.004238572440827675, 'получа': 0.016348779414621033, 'образован': 0.007152590993896701, 'акуш': 0.05722072795117361, 

In [ ]:
gc.collect()